## Read data

In [1]:
import pandas as pd
import csv
import os
import numpy as np
import datetime
from xlsxwriter.workbook import Workbook
from xlsxwriter import Workbook
import time
Working_dir = os.getcwd()

In [2]:
zabbix_df = pd.read_excel( os.path.join( Working_dir, "zabbix_v2_init.xlsx" ) )

In [3]:
lsf_df = pd.read_excel( os.path.join( Working_dir, "jobs_v2_init.xlsx" ))

In [4]:
# lsf_df = pd.read_excel( os.path.join( Working_dir, "jobs_v2_py.xlsx" ))
# lsf_df.shape[0]

Create Python date and time in comparable format (UTC)

In [5]:
print(zabbix_df.columns.values)
print(lsf_df.columns.values)

['Start_Timestamp (1h interval)' 'min_power' 'avg_power' 'max_power'
 'measure time in seconds']
['id' 'jobid' 'number of cores used' 'user name' 'queue name'
 'directory of running executable' 'name of executable' 'job status'
 'start unixtime' 'stop unixtime' 'numhost' 'Unnamed: 11' 'from']


In [6]:
zabbix_df["date_py"] = zabbix_df["Start_Timestamp (1h interval)"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).date())
zabbix_df["time_py"] = zabbix_df["Start_Timestamp (1h interval)"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).time())
zabbix_df["timestamp_py"] = zabbix_df["Start_Timestamp (1h interval)"].apply(lambda x: datetime.datetime.utcfromtimestamp(x))
# lsf_df["start_date_py"] = lsf_df["start unixtime"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).date())
# lsf_df["start_time_py"] = lsf_df["start unixtime"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).time())
# lsf_df["end_date_py"] = lsf_df["stop unixtime"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).date())
# lsf_df["end_time_py"] = lsf_df["stop unixtime"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).time())
lsf_df["start_timestamp_py"] = lsf_df["start unixtime"].apply(lambda x: datetime.datetime.utcfromtimestamp(x))
lsf_df["stop_timestamp_py"] = lsf_df["stop unixtime"].apply(lambda x: datetime.datetime.utcfromtimestamp(x))
lsf_df['directory of running executable'] = lsf_df['directory of running executable'].fillna('')
lsf_df['name of executable'] = lsf_df['name of executable'].fillna('')

In [7]:
zabbix_df['energy_avg (kWh)']=zabbix_df['avg_power'].copy()*zabbix_df['measure time in seconds'].copy()/60000.
zabbix_df['energy_min (kWh)']=zabbix_df['min_power'].copy()*zabbix_df['measure time in seconds'].copy()/60000.
zabbix_df['energy_max (kWh)']=zabbix_df['max_power'].copy()*zabbix_df['measure time in seconds'].copy()/60000.

In [8]:
# zabbix_df.drop( ['Start_Timestamp (1h interval)'], axis=1, inplace=True )
zabbix_df.head()

,Start_Timestamp (1h interval),min_power,avg_power,max_power,measure time in seconds,date_py,time_py,timestamp_py,energy_avg (kWh),energy_min (kWh),energy_max (kWh)
0,1487505600,38600,40637,41500,60.00,2017-02-19,12:00:00,2017-02-19 12:00:00,40.637000,38.600000,41.500000
1,1487509200,40010,40863,41530,60.00,2017-02-19,13:00:00,2017-02-19 13:00:00,40.863000,40.010000,41.530000
2,1487512800,37740,39971,41430,59.75,2017-02-19,14:00:00,2017-02-19 14:00:00,39.804454,37.582750,41.257375
3,1487516400,36190,38517,39470,60.25,2017-02-19,15:00:00,2017-02-19 15:00:00,38.677487,36.340792,39.634458
4,1487520000,38330,38735,39200,60.00,2017-02-19,16:00:00,2017-02-19 16:00:00,38.735000,38.330000,39.200000


#### All job ids are unique, they cannot be used as an identifier (it is checked for one month below)
#### Different processes names can be identifiers, so we form the full names in lsf table 
Number of different job names ("folder+name")

In [9]:
lsf_df['executable_full_name'] = lsf_df['directory of running executable'] + lsf_df['name of executable']

In [10]:
print('Unique names of jobs in a year: ',len(lsf_df.executable_full_name.unique()))
print('Jobs done in a year (including repetitions of the same jobs): ', lsf_df.shape[0])

Unique names of jobs in a year:  4614
Jobs done in a year (including repetitions of the same jobs):  538837


User names

In [11]:
print('Unique user names in a year: ',len(lsf_df['user name'].unique()))

Unique user names in a year:  119


In [12]:
# l = list(lsf_df.executable_full_name)
# dict((x,l.count(x)) for x in l)

In [13]:
lsf_df.drop( ['Unnamed: 11', 'from', 'id', 'numhost','directory of running executable','name of executable'], axis=1, inplace=True )
# lsf_df.drop( [ 'start unixtime', 'stop unixtime'], axis=1, inplace=True )
lsf_df.head()

,jobid,number of cores used,user name,queue name,job status,start unixtime,stop unixtime,start_timestamp_py,stop_timestamp_py,executable_full_name
0,967084,32,gusso,cresco4_h6,DONE,1483190848,1483224146,2016-12-31 13:27:28,2016-12-31 22:42:26,PFS/porq1_1M/SiH/Grossman/cp2kcp2k_launch.sh
1,967085,32,gusso,cresco4_h6,DONE,1483190848,1483224605,2016-12-31 13:27:28,2016-12-31 22:50:05,PFS/porq1_1M/SiH/Grossman/cp2kcp2k_launch.sh
2,967086,32,gusso,cresco4_h6,DONE,1483190848,1483224688,2016-12-31 13:27:28,2016-12-31 22:51:28,PFS/porq1_1M/SiH/Grossman/cp2kcp2k_launch.sh
3,967087,32,gusso,cresco4_h6,DONE,1483190848,1483224792,2016-12-31 13:27:28,2016-12-31 22:53:12,PFS/porq1_1M/SiH/Grossman/cp2kcp2k_launch.sh
4,967088,32,gusso,cresco4_h6,DONE,1483190848,1483224979,2016-12-31 13:27:28,2016-12-31 22:56:19,PFS/porq1_1M/SiH/Grossman/cp2kcp2k_launch.sh


In [14]:
print(zabbix_df.columns.values, "\n")
print(lsf_df.columns.values)

['Start_Timestamp (1h interval)' 'min_power' 'avg_power' 'max_power'
 'measure time in seconds' 'date_py' 'time_py' 'timestamp_py'
 'energy_avg (kWh)' 'energy_min (kWh)' 'energy_max (kWh)'] 

['jobid' 'number of cores used' 'user name' 'queue name' 'job status'
 'start unixtime' 'stop unixtime' 'start_timestamp_py' 'stop_timestamp_py'
 'executable_full_name']


In [15]:
print(zabbix_df.shape)
print(lsf_df.shape)

(8763, 11)
(538837, 10)


In [ ]:
excel_writer_zabbix = pd.ExcelWriter( os.path.join( Working_dir, "zabbix_v2_py.xlsx"))
zabbix_df.to_excel(excel_writer_zabbix)
excel_writer_zabbix.save()

In [16]:
lsf_df = lsf_df.sort_values('stop_timestamp_py')

In [17]:
excel_writer_lsf = pd.ExcelWriter( os.path.join( Working_dir, "lsf_v2_py.xlsx" ))
# excel_writer_lsf = pd.ExcelWriter( "lsf_v2_py.xlsx" )
lsf_df.to_excel(excel_writer_lsf)
excel_writer_lsf.save()

---------------

### Extract data for 19, February, 2017, 12:00 - 19, March, 2017, 12:00

In lsf (jobs) file recordings starts from 2017-02-19 12:00:00, we need an interseciton of recordings from Lsf and Zabbix

Find jobs that finished after 2017-02-19 12:00, started from this time, ended after and started before 2017-03-19 12:00

In [ ]:
lsf_month_df = pd.DataFrame(columns = lsf_df.columns.values)
i = 0
for index, row in lsf_df.iterrows():
#     start_datetime = datetime.datetime.utcfromtimestamp(row["start unixtime"])
    start_datetime = datetime.datetime.combine(row.start_date_py, \
                                           row.start_time_py )
    end_datetime   = datetime.datetime.combine(row.end_date_py, \
                                           row.end_time_py )
    #datetime.datetime.utcfromtimestamp(row["stop unixtime"])
        
    moment_left = datetime.datetime(2017,2,19, 12,0)
    moment_right = datetime.datetime(2017,3,19, 12,0)
    
    if ( (start_datetime >= moment_left  and end_datetime   <= moment_right) or\
         (start_datetime <= moment_left  and end_datetime   >= moment_left) or\
         (start_datetime <= moment_right and end_datetime  >= moment_right)):
        lsf_month_df.loc[i] = row
        i+=1
    if (start_datetime >= datetime.datetime(2017,4,19, 0,0)):
        break

In [ ]:
zabbix_month_df = pd.DataFrame(columns = zabbix_df.columns.values)
i = 0
zabbix_timestamp = datetime.datetime.utcfromtimestamp(zabbix_df.loc[0,\
                                                                    "Start_Timestamp (1h interval)"])
while (zabbix_timestamp < datetime.datetime(2017,3,19,12,0)):
    zabbix_month_df.loc[i] = zabbix_df.loc[i]
    i+=1
    zabbix_timestamp = datetime.datetime.utcfromtimestamp(zabbix_df.loc[i,\
                                                                    "Start_Timestamp (1h interval)"])

In [ ]:
# excel_writer_zabbix = pd.ExcelWriter( Working_dir+'\\zabbix_month_py.xlsx')
# zabbix_month_df.to_excel(excel_writer_zabbix)

# excel_writer_lsf = pd.ExcelWriter( Working_dir+'\\lsf_month_py.xlsx')
# lsf_month_df.to_excel(excel_writer_lsf)

In [ ]:
zabbix_month_df.head()

In [ ]:
lsf_month_df.tail()

In [ ]:
excel_writer_zabbix_month = pd.ExcelWriter( 'zabbix_month_py.xlsx')
zabbix_month_df.to_excel(excel_writer_zabbix_month)

excel_writer_lsf_month = pd.ExcelWriter( 'lsf_month_py.xlsx')
lsf_month_df.to_excel(excel_writer_lsf_month)

#### Job ids check up for one month. All job ids are different

In [ ]:
zabbix_month_df = pd.read_excel( os.path.join( Working_dir, "zabbix_month_py.xlsx" ) )
lsf_month_df = pd.read_excel( os.path.join( Working_dir, "lsf_month_py.xlsx" ) )

job_ids = list(lsf_month_df.jobid)
job_ids_dict = dict((x,job_ids.count(x)) for x in job_ids)
all(value == 1 for value in job_ids_dict.values())

------

### How many processes running every second
Processes are weighted during each second with the number of cores they require

-----------
Creating a dictionary

Date + hour starting - key of an upper level

Mins + sec - a key whithin an hour (on a lower level)

----------

### Month

Here we will simply add 1 to each second when the app was running

In [ ]:
zabbix_calc = zabbix_month_df.copy()
lsf_calc = lsf_month_df.copy()

In [ ]:
apps_each_sec = { zabbix_calc.timestamp_py.iloc[0] : {}}
print(type(apps_each_sec))
for i in range (0, zabbix_calc.shape[0]):
    
    apps_each_sec[zabbix_calc.timestamp_py.iloc[i]] = {zabbix_calc.timestamp_py.iloc[i] \
                                                                  + datetime.timedelta(0,0) : 0}
    for j in range (0,3600):
        apps_each_sec[zabbix_calc.timestamp_py.iloc[i]][zabbix_calc.timestamp_py.iloc[i] \
                                                                  + datetime.timedelta(0,j)] = 0

Here we will save the weighted number of tasks 

In [ ]:
apps_weighted_each_sec = { zabbix_calc.timestamp_py.iloc[0] : {}}
for i in range (0, zabbix_calc.shape[0]):
    
    apps_weighted_each_sec[zabbix_calc.timestamp_py.iloc[i]] = {zabbix_calc.timestamp_py.iloc[i] \
                                                                  + datetime.timedelta(0,0) : 0}
    for j in range (0,3600):
        apps_weighted_each_sec[zabbix_calc.timestamp_py.iloc[i]][zabbix_calc.timestamp_py.iloc[i] \
                                                                  + datetime.timedelta(0,j)] = 0

In [ ]:
max_cores = lsf_month_df['number of cores used'].max()
max_cores

In [ ]:
min_time = zabbix_calc.timestamp_py.iloc[0]
max_time = zabbix_calc.timestamp_py.iloc[zabbix_calc.shape[0]-1] + datetime.timedelta(minutes = 59, seconds = 59)
max_cores = lsf_calc['number of cores used'].max()

for index, row in lsf_calc.iterrows(): #.iloc[:1]
#     print(row)
#     print(type(row))
    start_time = datetime.datetime.combine(row.start_date_py, \
                                           row.start_time_py )
    end_time = datetime.datetime.combine(row.end_date_py, \
                                           row.end_time_py )
    cores = row['number of cores used']#/max_cores
#     print('cores',cores)
    t = start_time
    
    if min_time > start_time:
        t = min_time
    while t <= end_time and t <= max_time:
        start_in_table = t
        start_in_table = start_in_table.replace(minute=0, second=0)
#         print('start_in_table', start_in_table)
#         print('t', t)
        apps_each_sec[start_in_table][t] +=1
        apps_weighted_each_sec[start_in_table][t] += cores
        t += datetime.timedelta(0,1)

In [ ]:
np.save('apps_each_sec_month.npy',apps_each_sec ) 
np.save('apps_weighted_each_sec_month.npy',apps_weighted_each_sec ) 

### Year

In [ ]:
zabbix_calc = zabbix_df.copy()
lsf_calc = lsf_df.copy()

In [ ]:
np.save('apps_each_sec.npy',apps_each_sec ) 
np.save('apps_weighted_each_sec.npy',apps_weighted_each_sec ) 

----

In [ ]:
proc_df = pd.DataFrame(columns = lsf_month_df.executable_full_name.unique(), index=zabbix_month_df.timestamp_py)

In [ ]:
proc_df = pd.DataFrame(columns = lsf_month_df.executable_full_name.unique(), index=zabbix_month_df.timestamp_py)

min_time = zabbix_month_df.timestamp_py.iloc[0]
max_time = zabbix_month_df.timestamp_py.iloc[zabbix_month_df.shape[0]-1] + datetime.timedelta(minutes = 59, seconds = 59)
max_cores = lsf_month_df['number of cores used'].max()

proc_df[:] = 0 
i=0
# iterate over jobs in lsf
for index, line in lsf_month_df.iterrows():#.iloc[]
#     print(line)
    #i-line index
    i+=1
    job_name = line.executable_full_name
    cores = line['number of cores used']
    # start and end time in full date format 
    start_time = datetime.datetime.combine(line.start_date_py, \
                                           line.start_time_py )
    end_time = datetime.datetime.combine(line.end_date_py, \
                                           line.end_time_py )
    # t is our current position
    t = start_time
    # process length
    length = end_time - start_time + datetime.timedelta(seconds=1)
    # if a proces began before the first timestamp in zabbix
    # decrease length and step onto the first timestamp in zabbix
    if min_time > start_time:
        t = min_time
        length -= min_time - start_time
    # while current position is not at the end time of a process or end of table
    while t <= end_time and t <= max_time:
        # find row name in table = even hour and next row name
        start_in_table = t
        start_in_table = start_in_table.replace(minute=0, second=0)
        next_in_table = start_in_table + datetime.timedelta(hours=1)
        
        # if a process ends within this hour write its weight 
        # and put current position far to the right
        if ( t + length < next_in_table ):
#             print('length', length)
            proc_df.loc[start_in_table,job_name] += (length.total_seconds()/3600)*(cores)#/max_cores)
            t = next_in_table
        else:
            # if a process lasts longer than the current hour range
            # add number of seconds which it runs during this hour
            # decrease length of the process by the same mount
            # write its weight to the matrix
            to_add = next_in_table - t
            length -= to_add
            
            proc_df.loc[start_in_table,job_name] += (to_add.total_seconds()/3600)*(cores)#/max_cores)
            t += to_add
    if (i % 1000 == 0):
        print(i)

In [ ]:
proc_df.head()

In [ ]:
excel_writer_matrix = pd.ExcelWriter( 'proc_matrix_less_coeff_month.xlsx')
proc_df.to_excel(excel_writer_matrix)
excel_writer_matrix.save()

--------------
### Add max & min equations
- Find which sum of weights were max
- Find what apps were active this hour (add new line in new DataFrame, then only matrices indices will merge) and this max second
- Multiply weights by corresponding unknown variables == Put weights on the new lines

In [ ]:
# apps_each_sec = np.load('apps_each_sec.npy') 
apps_weighted_each_sec = np.load('apps_weighted_each_sec_month.npy') 

In [ ]:
# apps_each_sec = apps_each_sec[()]
apps_weighted_each_sec = apps_weighted_each_sec[()]

In [ ]:
proc_max_df = pd.DataFrame(columns = lsf_month_df.executable_full_name.unique())

proc_min_df = pd.DataFrame(columns = lsf_month_df.executable_full_name.unique())

# proc_max_df.loc[zabbix_month_df.timestamp_py.iloc[0]] = np.zeros(proc_max_df.shape[1])
for k,row in apps_weighted_each_sec.items():
    values_list = list(row.values())
    proc_max_df.loc[ list(row.keys())[values_list.index(max(values_list))] ] = np.zeros(proc_max_df.shape[1])
    proc_min_df.loc[ list(row.keys())[values_list.index(min(values_list))] ] = np.zeros(proc_min_df.shape[1])

In [ ]:
print(type(proc_df.index.values[0]))#==datetime.datetime(year=2017,month=2, day=19,hour=12, second=2))
print(datetime.datetime(year=2017,month=2, day=19,hour=12, second=2))

In [ ]:
min_time = zabbix_month_df.timestamp_py.iloc[0]
max_time = zabbix_month_df.timestamp_py.iloc[zabbix_month_df.shape[0]-1] + datetime.timedelta(minutes = 59, seconds = 59)
max_cores = lsf_month_df['number of cores used'].max()

proc_max_df[:] = 0 
proc_min_df[:] = 0 
i=0
# iterate over jobs in lsf
for index, line in lsf_month_df.iterrows():#.iloc[]
#     print(line)
    #i-line index
    i+=1
    job_name = line.executable_full_name
    cores = line['number of cores used']
    # start and end time in full date format 
    start_time = datetime.datetime.combine(line.start_date_py, \
                                           line.start_time_py )
    end_time = datetime.datetime.combine(line.end_date_py, \
                                           line.end_time_py )
    for index, row in proc_max_df.iterrows():
#         print(type(start_time.timestamp()))
#         print(type(index.timestamp()))
        if ((index.timestamp() >= start_time.timestamp()) & (index.timestamp() <= end_time.timestamp())):
#             print(start_time, index, end_time, '\n')
            proc_max_df.loc[index,job_name] += cores#/max_cores
        if (index.timestamp() > end_time.timestamp()):
            break
            
    for index, row in proc_min_df.iterrows():
#         print(type(start_time.timestamp()))
#         print(type(index.timestamp()))
        if ((index.timestamp() >= start_time.timestamp()) & (index.timestamp() <= end_time.timestamp())):
#             print(start_time, index, end_time, '\n')
            proc_min_df.loc[index,job_name] += cores#/max_cores
        if (index.timestamp() > end_time.timestamp()):
            break
#     print('after break in iterrows')
    if (i % 1000 == 0):
        print(i)

In [ ]:
excel_writer_matrix_max = pd.ExcelWriter( os.path.join( Working_dir, 'proc_max_matrix.xlsx' ))
proc_max_df.to_excel(excel_writer_matrix_max)
excel_writer_matrix_max.save()

excel_writer_matrix_min = pd.ExcelWriter( os.path.join( Working_dir, 'proc_min_matrix.xlsx' ))
proc_min_df.to_excel(excel_writer_matrix_min)
excel_writer_matrix_min.save()

-----
## Regrouped lsf dataset

In [74]:
lsf_gr_df = pd.read_excel( os.path.join( Working_dir, "Improved_lsf_for_jobs_running_less_than_30.xlsx" ) )

In [75]:
lsf_gr_df.columns.values

array(['jobid', 'inizio', 'fine', 'durata in sec', 'numcores', 'user',
       'queue', 'directory', 'executable', 'jobstatus', 'numhost'], dtype=object)

In [76]:
lsf_gr_df["start_timestamp_py"] = lsf_gr_df["inizio"].apply(lambda x: datetime.datetime.utcfromtimestamp(x))
lsf_gr_df["stop_timestamp_py"] = lsf_gr_df["fine"].apply(lambda x: datetime.datetime.utcfromtimestamp(x))
lsf_gr_df["duration_py"] = (lsf_gr_df["stop_timestamp_py"] - lsf_gr_df["start_timestamp_py"]).apply(lambda x: x.total_seconds())
lsf_gr_df['directory'] = lsf_gr_df['directory'].fillna('')
lsf_gr_df['executable'] = lsf_gr_df['executable'].fillna('')

### Check whether difference between stop and start time is exactly "durata"
python checkup

In [77]:
lsf_gr_df[lsf_gr_df["duration_py"] == lsf_gr_df["durata in sec"]].shape

(502272, 14)

In [78]:
lsf_gr_df[lsf_gr_df["duration_py"] != lsf_gr_df["durata in sec"]].shape

(69199, 14)

unix checkup

In [79]:
lsf_gr_df[(lsf_gr_df["fine"] - lsf_gr_df["inizio"]) != lsf_gr_df["durata in sec"]].shape

(0, 14)

#### We will keep duration_py = durata in sec, since unix time difference is more precise than python datetime difference

In [80]:
lsf_gr_df["duration_py"] = lsf_gr_df["durata in sec"].copy()

In [81]:
for i in range(lsf_gr_df.shape[0]):
    a = lsf_gr_df.loc[i, 'executable']
    if type(a)!=str and type(a) != int:
        print(lsf_gr_df.loc[i, ['jobid', 'executable']])
        print(type(a))
        print('\n')


jobid           502922
executable    03:00:00
Name: 467782, dtype: object
<class 'datetime.time'>


jobid           502923
executable    03:00:00
Name: 467783, dtype: object
<class 'datetime.time'>




### Create full path name for the new dataset

In [82]:
lsf_gr_df["executable_full_name"] = lsf_gr_df["directory"].apply( lambda x: str(x) if type(x) == int else x ) \
    + lsf_gr_df["executable"].apply( lambda x: str(x) if type(x) == int else \
                                    (x.strftime('%H-%M-%S') if type(x)==datetime.time else x) )

### Drop unnecessary columns

In [83]:
lsf_gr_df.drop( ['durata in sec'], axis=1, inplace=True )

In [84]:
lsf_gr_df = lsf_gr_df.sort_values('stop_timestamp_py')

In [85]:
lsf_gr_df.head()

,jobid,inizio,fine,numcores,user,queue,directory,executable,jobstatus,numhost,start_timestamp_py,stop_timestamp_py,duration_py,executable_full_name
544642,964821,1482748132,1482748136,64,grena,cresco3_h144,PFS/porq1_1M/Al_cluster,qe5.0.3_par,DONE,3,2016-12-26 10:28:52,2016-12-26 10:28:56,4,PFS/porq1_1M/Al_clusterqe5.0.3_par
544643,964823,1482748161,1482748168,64,grena,cresco3_h144,PFS/porq1_1M/Al_cluster,qe5.0.3_par,DONE,3,2016-12-26 10:29:21,2016-12-26 10:29:28,7,PFS/porq1_1M/Al_clusterqe5.0.3_par
544480,953307,1478885885,1482809980,12,aprea,cresco4_h144,,./assembly.sh,EXIT,1,2016-11-11 17:38:05,2016-12-27 03:39:40,3924095,./assembly.sh
544481,953308,1478885885,1482810053,12,aprea,cresco4_h144,,./assembly.sh,EXIT,1,2016-11-11 17:38:05,2016-12-27 03:40:53,3924168,./assembly.sh
544644,965254,1482834939,1482834945,64,grena,cresco3_h144,PFS/porq1_1M/Al_cluster,qe5.0.3_par,DONE,3,2016-12-27 10:35:39,2016-12-27 10:35:45,6,PFS/porq1_1M/Al_clusterqe5.0.3_par


### Read preprocessed lsf_py from initial dataset

In [33]:
lsf_py_df = pd.read_excel( os.path.join( Working_dir, "lsf_v2_py.xlsx" ))

### Short comparison of lengths and jobids

In [44]:
print(lsf_py_df.shape, lsf_gr_df.shape)

(538837, 10) (571471, 11)


In [87]:
print(lsf_gr_df.jobid.unique().shape[0], lsf_py_df.jobid.unique().shape[0])

571471 529551


### Check executable full path names in comparison with the initial dataset

In [100]:
print(lsf_gr_df.executable_full_name.isin( lsf_py_df.executable_full_name ).value_counts())

False    540661
True      30810
Name: executable_full_name, dtype: int64


### Read preprocessed zabbix df and crop new lsf dataset according to zabbix range

In [101]:
zabbix_py_df = pd.read_excel( os.path.join( Working_dir, "zabbix_v2_py.xlsx" ))

In [111]:
lsf_gr_df = lsf_gr_df.drop(lsf_gr_df[lsf_gr_df.stop_timestamp_py < zabbix_py_df.loc[0, 'timestamp_py']].index &\
                          lsf_gr_df[lsf_gr_df.start_timestamp_py > zabbix_py_df.loc[zabbix_py_df.shape[0]-1, 'timestamp_py']].index)

In [112]:
lsf_gr_df.shape

(571471, 14)

### Percentage of shortly finished applications

In [114]:
short_activity_time = lsf_gr_df[lsf_gr_df.duration_py <= 30 ].shape[0]

In [115]:
print(short_activity_time/lsf_gr_df.shape[0]*100)

20.619419008138646


## Observations on the new dataset
- The new dataset is larger than the initial one
- Names of executables sometimes contain numbers (20, 180, etc) or even timestamp (03:00:00)
- If I understood correctly, after regrouping, start time is minimum of the start times of all grouped lines with the same jobid, same for stop time, but it is maximum of all corresponding lines. If we take min-max, we lose data on applications with shorter activity time. Why do we need regrouping?
- Should I recalculate everything with this new lsf dataset?
- Should I consider jobs <=30 sec duration for each month or for the whole year? I prefer the first, for each month, for consistency.